# Network Analysis Project
## Network Part

#### Prepare Data

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('MTA_Subway_Stations_20241103.csv')[['Station ID','Daytime Routes','GTFS Latitude','GTFS Longitude']]
df.head()

,Station ID,Daytime Routes,GTFS Latitude,GTFS Longitude
0,1,N W,40.775036,-73.912034
1,2,N W,40.770258,-73.917843
2,3,N W,40.766779,-73.921479
3,4,N W,40.761820,-73.925508
4,5,N W,40.756804,-73.929575


In [3]:
from math import sqrt

def distance(latA,longA,latB,longB):
    return sqrt((latA-latB)**2+(longA-longB)**2)

def distance_stations(stationA,stationB,df):
    return distance(df['GTFS Latitude'][stationA],df['GTFS Longitude'][stationA],df['GTFS Latitude'][stationB],df['GTFS Longitude'][stationB])

distance (df['GTFS Latitude'][0],df['GTFS Longitude'][0],df['GTFS Latitude'][1],df['GTFS Longitude'][1]), distance_stations(0,1,df)

(0.007521553363501997, 0.007521553363501997)

#### Seperate train lines

In [86]:
routes_to_station_index = dict()
for index in range(df.shape[0]):
    stations = df['Daytime Routes'][index].strip().split(' ')
    for s in stations:
        if not (s in routes_to_station_index.keys()):
            routes_to_station_index[s] = []
        routes_to_station_index[s].append(index)

In [87]:
routes_to_station_index.pop('SIR')
routes_to_station_index.pop('S')
all_routes = routes_to_station_index.keys()
all_routes

dict_keys(['N', 'W', 'R', 'Q', 'B', 'D', 'F', 'J', 'Z', 'M', 'L', 'A', 'C', 'E', 'G', '1', '2', '3', '4', '5', '6', '7'])

In [88]:
edges_in_routes = dict()
for route in all_routes:
    edges_in_routes[route] = []
    for stationA in routes_to_station_index[route]:
        for stationB in routes_to_station_index[route]:
            if stationA < stationB:
                edges_in_routes[route].append((stationA,stationB,distance_stations(stationA,stationB,df)))
                
edges_in_routes.keys()

dict_keys(['N', 'W', 'R', 'Q', 'B', 'D', 'F', 'J', 'Z', 'M', 'L', 'A', 'C', 'E', 'G', '1', '2', '3', '4', '5', '6', '7'])

#### Calculate Number of Running Trains

In [81]:
df2 = pd.read_csv('MTA_Service_2024.csv')
part1 = df2[df2['line']=='1']
part1.head()

,month,division,line,day_type,num_sched_trains,num_actual_trains,service delivered
0,2024/9/1,A DIVISION,1,1.0,1680.0,1610.0,95.833330
1,2024/9/1,A DIVISION,1,2.0,1410.0,1365.0,96.808510
48,2024/8/1,A DIVISION,1,1.0,1848.0,1763.0,95.400435
49,2024/8/1,A DIVISION,1,2.0,1296.0,1249.0,96.373457
97,2024/7/1,A DIVISION,1,1.0,1848.0,1738.0,94.047620


In [94]:
running_trains = dict()
for route in all_routes:
    if route == 'J' or route == 'Z':
        part = df2[df2['line']=='JZ']
        running_trains[route] = part['num_actual_trains'].sum()/2
    else:
        part = df2[df2['line']==route]
        running_trains[route] = part['num_actual_trains'].sum()

In [95]:
running_trains

{'N': 26203.0,
 'W': 6672.0,
 'R': 29612.0,
 'Q': 30970.0,
 'B': 14148.0,
 'D': 27793.0,
 'F': 34261.0,
 'J': 8173.0,
 'Z': 8173.0,
 'M': 19249.0,
 'L': 28722.0,
 'A': 35413.0,
 'C': 23599.0,
 'E': 19445.0,
 'G': 21222.0,
 '1': 24266.0,
 '2': 32532.0,
 '3': 28360.0,
 '4': 42216.0,
 '5': 30299.0,
 '6': 28373.0,
 '7': 33304.0}

#### Find route for each line using MST

In [89]:
class DisjointSet:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, u):
        if self.parent[u] != u:
            self.parent[u] = self.find(self.parent[u])
        return self.parent[u]

    def union(self, u, v):
        rootU = self.find(u)
        rootV = self.find(v)
        
        if rootU != rootV:
            if self.rank[rootU] > self.rank[rootV]:
                self.parent[rootV] = rootU
            elif self.rank[rootU] < self.rank[rootV]:
                self.parent[rootU] = rootV
            else:
                self.parent[rootV] = rootU
                self.rank[rootU] += 1

def kruskal_mst(nodes, edges):
    # Find the largest node index
    max_node = max(max(u, v) for u, v, weight in edges) + 1
    # Initialize disjoint set with size based on largest node index
    ds = DisjointSet(max_node)
    # Sort edges by weight
    edges = sorted(edges, key=lambda x: x[2])
    
    mst = []
    mst_weight = 0

    for u, v, weight in edges:
        # Find roots of the sets for u and v
        if ds.find(u) != ds.find(v):
            # If they are in different sets, include this edge in the MST
            mst.append((u, v, weight))
            mst_weight += weight
            ds.union(u, v)
    
    return mst, mst_weight


In [106]:
edges_mst_in_routes=dict()
all_edges = []
for route in all_routes:
    edges = edges_in_routes[route]
    edges = sorted(edges, key=lambda x: x[2])
    nodes = routes_to_station_index[route]
    edges_mst, mst_weight = kruskal_mst(nodes,edges)
    edges_mst_new=[]
    for edge in edges_mst:
        # Combine distance with running trains
        edges_mst_new.append((edge[0],edge[1],edge[2]*1000000/(running_trains[route]/len(routes_to_station_index[route]))))
    edges_mst_in_routes[route]=edges_mst_new
    all_edges += edges_mst_new



In [107]:
all_edges

[(77, 78, 5.169558791645291),
 (1, 2, 5.377403514784622),
 (4, 5, 5.3954805052580195),
 (10, 11, 5.602808344232903),
 (9, 10, 6.2473706762654215),
 (8, 9, 6.304035673173611),
 (73, 74, 6.729904629725273),
 (2, 3, 6.827592123515896),
 (74, 75, 6.8758538708848596),
 (3, 4, 6.900473961544373),
 (6, 7, 6.900637532669199),
 (76, 77, 7.013756571370475),
 (72, 73, 7.755046284781335),
 (7, 8, 7.813966926829023),
 (70, 71, 7.858176412253314),
 (0, 1, 8.037380993705145),
 (5, 462, 8.328606123043915),
 (75, 76, 8.583210855100914),
 (34, 70, 9.415338262481903),
 (71, 72, 11.889023414577244),
 (11, 14, 15.041965364014489),
 (57, 78, 16.660165627024973),
 (31, 34, 21.247006347885623),
 (14, 18, 21.34428629539444),
 (6, 462, 31.661452171585807),
 (26, 31, 40.34540400703283),
 (18, 26, 43.72036509839881),
 (11, 12, 14.271087288226404),
 (21, 22, 14.297867897203291),
 (20, 21, 14.312770717998264),
 (12, 13, 14.621708532856445),
 (19, 20, 16.619710973587164),
 (1, 2, 17.347520548838073),
 (4, 5, 17.4058

In [108]:
df.shape[0]

496

In [109]:
import networkx as nx

G = nx.Graph()
all_nodes = list(range(df.shape[0]))
G.add_nodes_from(all_nodes)

In [110]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Generate a color map
num_groups = len(all_routes)
colors = cm.get_cmap('tab10', num_groups)  # 'tab10' has distinct colors

i=0
# Add edges, assigning each group a color from the colormap
for route in all_routes:
    edges = edges_mst_in_routes[route]
    color = colors(i)  # Get a color from the colormap
    G.add_weighted_edges_from(edges, color=color)
    i+=1

# Extract the color for each edge to use in the plot
edge_colors = [G[u][v]['color'] for u, v in G.edges]



# Draw the graph
pos = nx.kamada_kawai_layout(G)
plt.figure(figsize=(150,100))  # Set a larger figure size
nx.draw(G, pos, edge_color=edge_colors, node_size=500,with_labels=True,font_size=8, font_weight='bold')
# nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): G[u][v]['weight'] for u, v in G.edges})
plt.savefig('network2.jpg')

C:\Users\Yizhou\AppData\Local\Temp\ipykernel_13296\2938167897.py:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = cm.get_cmap('tab10', num_groups)  # 'tab10' has distinct colors
